In [63]:
import pandas as pd

In [64]:
def prepare(data):
    data['import.start'] = pd.to_datetime(data['import.start'])
    data['import.stop'] = pd.to_datetime(data['import.stop'])
    data['created'] = pd.to_datetime(data['created'])
    data['started'] = pd.to_datetime(data['started'])
    data['finished'] = pd.to_datetime(data['finished'])
    
    data['starttime'] = (data['started'] - data['created']).dt.total_seconds() 
    data['dockertime'] = (data['finished'] - data['started']).dt.total_seconds() 
    data['importtime'] = (data['import.stop'] - data['import.start']).dt.total_seconds() 
    
    started_temp = pd.DataFrame()
    docker_temp = pd.DataFrame()
    import_temp = pd.DataFrame()
    
    for i in range(8,29,5):
        started_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+1]) - pd.to_datetime(data.iloc[:, i])).dt.total_seconds() , True)
        docker_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+2]) - pd.to_datetime(data.iloc[:, i+1])).dt.total_seconds() , True)
        import_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+4]) - pd.to_datetime(data.iloc[:, i+3])).dt.total_seconds() , True)
        
    
    started_temp.columns = ['starttime{}'.format(i) for i in range(1, 6)]
    docker_temp.columns = ['dockertime{}'.format(i) for i in range(1, 6)]
    import_temp.columns = ['importtime{}'.format(i) for i in range(1, 6)]
    
    data = pd.concat([data, started_temp, docker_temp, import_temp], axis=1)
    
    return data

In [65]:
def extract_imports(imp):
    imp = imp.split("'")
    l = ""
    
    for i in imp:
        if i == "(" or i == ")" or i == ",)" or i == "," or i == "()" or i == ", ":
            continue
        else:
            if l == "":
                l = i
            else:
                l = l + ";" + i
    return l

This is one experiment run with using the top 10 packages over all functions
The installationtime is set to minus -1 if there was no install as for native python packages as os

In [66]:
#approach = "all_top_10"
#approach = "vanilla_approach"
approach = "cluster_top_10"
col = ['imports', 'buldingtime', 'installtime', 'starttime', 'dockertime',
       'importtime', 'starttime1', 'starttime2', 'starttime3', 'starttime4',
       'starttime5', 'dockertime1', 'dockertime2', 'dockertime3',
       'dockertime4', 'dockertime5', 'importtime1', 'importtime2',
       'importtime3', 'importtime4', 'importtime5']

In [67]:
data = pd.read_csv("data/"+approach+"/run10.csv")
data = prepare(data)
imp = data[col[0]]
data = data[col[1:]]

In [68]:
for i in range(9,0,-1):
    tmp = pd.read_csv("data/"+approach+"/run0"+str(i)+".csv")
    tmp = prepare(tmp)
    tmp = tmp[col[1:]]
    data = (data + tmp)
data = data/10
data.insert(0, col[0],imp)

/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_8026/337023197.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['created'] = pd.to_datetime(data['created'])


In [69]:
data.head()

,imports,buldingtime,installtime,starttime,dockertime,importtime,starttime1,starttime2,starttime3,starttime4,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
0,argparse;copy;datetime;funcx;itertools;json;ma...,15.60,12.19,2.180410,9.054598,7.674799,0.791878,0.840661,0.908933,0.730136,...,1.554564,1.552487,1.557682,1.554612,1.550625,0.444415,0.440468,0.442420,0.447363,0.444389
1,argparse;copy;funcx;itertools;json;matplotlib;...,15.16,11.58,2.278498,9.340353,7.962382,0.776983,0.753207,0.744361,0.698806,...,1.550500,1.548959,1.550702,1.547994,1.551501,0.440383,0.443808,0.443245,0.442713,0.441791
2,argparse;sys,9.57,6.23,2.267264,1.268426,0.027284,0.781053,0.766173,0.768329,0.784168,...,1.036251,1.035948,1.036531,1.036197,1.036698,0.006813,0.007098,0.006366,0.006247,0.006893
3,CLEAN,7.97,5.09,2.066498,1.207408,0.018558,0.692933,0.680355,0.647873,0.637311,...,1.021332,1.021416,1.020979,1.021203,1.021200,0.000221,0.000419,0.000387,0.000352,0.000368
4,CLEAN;os,8.20,4.93,2.274209,1.226708,0.015460,0.827641,0.871029,0.763014,0.738976,...,1.022042,1.020680,1.021262,1.021338,1.020796,0.000344,0.000279,0.000340,0.000340,0.000368


This is the mapping between imports and function
For this merge and the following ones, there is no need to think about on which join-operation. I cleaned the data beforehand. That is, there will be always a pair 
```python
pd.merge(func_mapping,data, on='imports')
```

In [70]:
func_mapping = pd.read_csv('experiments/function_mapping.csv')
func_mapping = func_mapping.drop(2)
func_mapping['func4.imports'] = func_mapping[func_mapping['func4.imports'].str.len() >= 0]['func4.imports'].map(lambda x: extract_imports(x))
func_mapping = func_mapping[func_mapping['func4.imports'] != ""]
func_mapping.rename(columns={'func4.imports': 'imports'}, inplace=True)
func_mapping.drop(columns=['Unnamed: 0'], inplace=True)

In [71]:
func_mapping.head()

,Unnamed: 0.1,func4.name,imports,Cluster
0,0,0-04-_check_slurm_job_statuses.py,subprocess,Job
3,3,873-04-call_mtrack.py,mtrack;sys,Data:_mtrack
4,4,43-04-make_corr_plots.py,json;os;xpcs_webplot,Sci:_Instruments
6,6,3-04-submit_slurm_job.py,os;subprocess,Job
7,7,397-01-rejection_sample.py,random,Trivial


Here is the mapping between functions and task
```python
pd.merge(func_mapping,task_mapping, on='func4.name')
```

In [72]:
task_mapping = pd.read_csv("funcx/task_function_mapping.csv")

In [73]:
task_mapping.head()

,task_uuid,func4.name,executiontime
0,000001d5-b3f0-4c17-8074-04f013f7bf52,0-04-_check_slurm_job_statuses.py,0.0243
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,0-04-_check_slurm_job_statuses.py,0.0331
2,00002642-f850-42b4-9e1e-2698c3396498,0-04-_check_slurm_job_statuses.py,0.0432
3,00002deb-cf79-43ed-8aff-3281938809fe,0-04-_check_slurm_job_statuses.py,0.0268
4,0000754b-898e-43dc-9b55-35d1a6fe6633,0-04-_check_slurm_job_statuses.py,0.0284


Here we have the timestamp for each task
For merging
```python
pd.merge(task_mapping,task_mapping, on='task_uuid')
```

In [74]:
task = pd.read_csv("funcx/tasks_pt1.csv")
task = pd.concat([task, pd.read_csv("funcx/tasks_pt2.csv")], ignore_index=True)
task['received'] = task['received'].interpolate()

In [75]:
task.head()

,task_uuid,user_id,endpoint_uuid,received
0,000001d5-b3f0-4c17-8074-04f013f7bf52,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.681417e+18
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679386e+18
2,00002642-f850-42b4-9e1e-2698c3396498,286.0,cbc1f8d4-71b2-4a41-8ed4-9c75421eb89f,1.683246e+18
3,00002deb-cf79-43ed-8aff-3281938809fe,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.677897e+18
4,0000754b-898e-43dc-9b55-35d1a6fe6633,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679277e+18


Here are information about the endpoints
For merging:
```python
pd.merge(task,endpoints, on='endpoint_uuid')
```

In [76]:
endpoints = pd.read_csv("funcx/endpoints.csv")

In [77]:
endpoints.head()

,endpoint_uuid,X_type,ip,cluster
0,00000000-0000-4000-8000-000011110000,LocalProvider,216.80.70.252,6
1,001fecc2-1393-4bd9-950e-5645e15e3a53,SlurmProvider,141.142.140.194,10
2,006f558d-df82-45c2-b2d5-94274ef41a69,SlurmProvider,141.142.140.194,10
3,00824163-569c-4f0d-953d-097cd3b7179d,NaN,159.226.43.47,10
4,01a3324e-7ad9-4563-b598-0c2499068a39,LocalProvider,128.135.204.6,8


Example merging in the end:
```python
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')
```

In [78]:
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')
r.to_csv("my_data.csv")

In [79]:
sorted_df = r.sort_values(by=["received"], ascending=True)
sorted_df = sorted_df.dropna()
sorted_df.to_csv(approach+"_data_sorted.csv")

In [80]:
sorted_df.head()

,task_uuid,user_id,endpoint_uuid,received,func4.name,executiontime,Unnamed: 0.1,imports,Cluster,buldingtime,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
1109608,2dda391f-cb85-4e2f-8ae7-c9c603a821dd,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669597e+18,296-04-tar.py,0.0021,436,os;pathlib;tarfile,Sci:_Instruments,7.7,...,1.047234,1.047545,1.047907,1.04703,1.048197,0.01628,0.016247,0.016803,0.016089,0.016373
1109630,ac3f3e5e-aeff-4236-a3a5-223dcf00aba4,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669598e+18,296-04-tar.py,0.0017,436,os;pathlib;tarfile,Sci:_Instruments,7.7,...,1.047234,1.047545,1.047907,1.04703,1.048197,0.01628,0.016247,0.016803,0.016089,0.016373
1109604,1347620d-56a1-4d30-b8d3-32ded9e08b47,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669598e+18,296-04-tar.py,0.0020,436,os;pathlib;tarfile,Sci:_Instruments,7.7,...,1.047234,1.047545,1.047907,1.04703,1.048197,0.01628,0.016247,0.016803,0.016089,0.016373
1109605,1fbeacd4-b3e9-4bb9-bc1c-92fc9abd238f,600.0,674446c2-4dc6-41a6-b767-bf6ce67df702,1.669598e+18,296-04-tar.py,0.0095,436,os;pathlib;tarfile,Sci:_Instruments,7.7,...,1.047234,1.047545,1.047907,1.04703,1.048197,0.01628,0.016247,0.016803,0.016089,0.016373
1109606,285a505d-66ae-4e5c-88a3-e70794e425ae,600.0,674446c2-4dc6-41a6-b767-bf6ce67df702,1.669598e+18,296-04-tar.py,0.0081,436,os;pathlib;tarfile,Sci:_Instruments,7.7,...,1.047234,1.047545,1.047907,1.04703,1.048197,0.01628,0.016247,0.016803,0.016089,0.016373


In [83]:
p = sorted_df.dropna()

In [84]:
p[p.task_uuid.isna()]

,task_uuid,user_id,endpoint_uuid,received,func4.name,executiontime,Unnamed: 0.1,imports,Cluster,buldingtime,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
